<a href="https://colab.research.google.com/github/unfreaked/Projects/blob/main/Travelitenerary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community langchain langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.8 MB/s eta 0:00:00


In [ ]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Initialize the LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",  # Add your Groq API key here
    model_name="llama-3.3-70b-versatile"
)

# Define the itinerary prompt
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])

class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    city: str
    interests: List[str]
    itinerary: str

def input_city(state: PlannerState) -> PlannerState:
    print("Please enter the city you want to visit for your day trip: ")
    user_message = input("Your Input: ")
    return {
        **state,
        "city": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

def input_interest(state: PlannerState) -> PlannerState:
    print(f"Please enter your interests for the trip to {state['city']} (comma-separated): ")
    user_message = input("Your Input: ")
    return {
        **state,
        "interests": [interest.strip() for interest in user_message.split(",")],
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

def create_itinerary(state: PlannerState) -> PlannerState:
    print(f"\nCreating an itinerary for {state['city']} based on interests: {', '.join(state['interests'])}")
    response = llm.invoke(itinerary_prompt.format_messages(
        city=state['city'],
        interests=", ".join(state['interests'])
    ))
    print("\nFinal Itinerary: ")
    print(response.content)
    return {
        **state,
        "itinerary": response.content,
        "messages": state['messages'] + [AIMessage(content=response.content)]
    }

def travel_planner(user_request: str):
    print(f"Initial Request: {user_request}\n")

    # Initialize state with all required fields
    state = {
        "messages": [HumanMessage(content=user_request)],
        "city": "",
        "interests": [],
        "itinerary": ""
    }

    # Execute the workflow steps sequentially
    state = input_city(state)
    state = input_interest(state)
    state = create_itinerary(state)

    return state['itinerary']

# Run the planner
user_request = "I want to plan a day trip"
travel_planner(user_request)

Initial Request: I want to plan a day trip

Please enter the city you want to visit for your day trip: 
Your Input: kolkata
Please enter your interests for the trip to kolkata (comma-separated): 
Your Input: eat

Creating an itinerary for kolkata based on interests: eat

Final Itinerary: 
For a food-focused day trip in Kolkata, consider the following itinerary:

* 9:00 AM: Start with a traditional Bengali breakfast at Putiram Sweets, trying their famous 'jalebi' and 'radhaballabi'
* 11:00 AM: Visit the College Street area and try some popular street food like 'kathi rolls' and 'phuchka'
* 1:00 PM: Head to Peter Cat for their signature 'chelo kebabs' and other Bengali dishes
* 3:00 PM: Take a break and indulge in some sweet treats at Balaram Mullick & Radharaman Mullick, known for their 'mishti doi' and 'sandesh'
* 6:00 PM: End the day with a visit to 6 Ballygunge Place, a restaurant serving authentic Bengali cuisine, and try their 'bhapa ilish' and 'kosha mangsho'


"For a food-focused day trip in Kolkata, consider the following itinerary:\n\n* 9:00 AM: Start with a traditional Bengali breakfast at Putiram Sweets, trying their famous 'jalebi' and 'radhaballabi'\n* 11:00 AM: Visit the College Street area and try some popular street food like 'kathi rolls' and 'phuchka'\n* 1:00 PM: Head to Peter Cat for their signature 'chelo kebabs' and other Bengali dishes\n* 3:00 PM: Take a break and indulge in some sweet treats at Balaram Mullick & Radharaman Mullick, known for their 'mishti doi' and 'sandesh'\n* 6:00 PM: End the day with a visit to 6 Ballygunge Place, a restaurant serving authentic Bengali cuisine, and try their 'bhapa ilish' and 'kosha mangsho'"

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
pip install pycountry-convert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 17.2 MB/s eta 0:00:00


In [ ]:
pip install pycountry-convert gradio langchain-groq

In [ ]:
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import random
import pycountry_convert as pc

# Initialize the LLM
llm = ChatGroq(
    temperature=0.7,
    groq_api_key="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",  # Add your Groq API key here
    model_name="llama-3.3-70b-versatile"
)

# Enhanced country to currency mapping with Indian cities
COUNTRY_CURRENCIES = {
    'united states': ('USD', '$'),
    'usa': ('USD', '$'),
    'france': ('EUR', '€'),
    'japan': ('JPY', '¥'),
    'united kingdom': ('GBP', '£'),
    'uk': ('GBP', '£'),
    'india': ('INR', '₹'),
    'indian': ('INR', '₹'),
    'delhi': ('INR', '₹'),
    'mumbai': ('INR', '₹'),
    'bangalore': ('INR', '₹'),
    'chennai': ('INR', '₹'),
    'kolkata': ('INR', '₹'),
    'hyderabad': ('INR', '₹'),
    'australia': ('AUD', 'A$'),
    # ... rest of the original mappings
}

def get_currency_for_country(country_name):
    """Get currency code and symbol for a given country name"""
    country_name = country_name.lower().strip()

    # First check our common currencies
    if country_name in COUNTRY_CURRENCIES:
        return COUNTRY_CURRENCIES[country_name]

    # Special handling for India
    if 'india' in country_name or 'indian' in country_name:
        return ('INR', '₹')

    # Try to get continent as fallback
    try:
        country_code = pc.country_name_to_country_alpha2(country_name, cn_name_format="default")

        # Explicit check for India by country code
        if country_code == 'IN':
            return ('INR', '₹')

        continent_code = pc.country_alpha2_to_continent_code(country_code)

        if continent_code == 'EU':
            return ('EUR', '€')
        elif continent_code == 'AS':
            return ('USD', '$')  # For other Asian countries
        elif continent_code in ['AF', 'SA']:
            return ('USD', '$')
        else:
            return ('USD', '$')
    except:
        # Final check for India in the name
        if 'india' in country_name.lower():
            return ('INR', '₹')
        return ('USD', '$')

# ... (rest of the code remains the same)

def extract_country_from_input(location_input):
    """Extract country from city, country format or return input as country"""
    if ',' in location_input:
        parts = [p.strip() for p in location_input.split(',')]
        return parts[-1]  # Return the country part
    return location_input  # If no comma, treat whole input as country

# Enhanced prompt with dynamic currency
def create_itinerary_prompt(city: str, interests: str, currency_code: str, currency_symbol: str):
    return ChatPromptTemplate.from_messages([
        ("system", f"""Create a detailed day trip itinerary for {city} based on these interests: {interests}.

         **Key Requirements:**
         - All prices must be in local currency: {currency_code} ({currency_symbol})
         - Format costs appropriately (e.g., {currency_symbol}100-{currency_symbol}200)
         - Include realistic price ranges for all activities/meals
         - Provide budget estimates for each time period
         - Mention if credit cards are widely accepted or if cash is preferred

         **Formatting Guidelines:**
         - Time blocks with emoji icons (🌅 Morning/☀️ Afternoon/🌙 Evening)
         - Bold section headers for each time period
         - Bullet points with relevant emojis
         - Exact location names with addresses
         - Cost estimates: {currency_symbol}50-{currency_symbol}100 style
         - Travel time between locations
         - Local money-saving tips
         - Payment method recommendations

         **Example for Japan:**
         - 🍣 Lunch at Sushi Dai: ¥2,000-¥3,500 (cash only)
         - 🚇 Subway ticket: ¥200 per ride (IC card recommended)

         **Example for France:**
         - 🥐 Breakfast at Café de Flore: €8-€15 (cards accepted)
         - 🎟 Louvre Museum entry: €17 (book online to skip queue)"""),
        ("human", "Create my personalized itinerary with local pricing!")
    ])

def generate_itinerary(city: str, interests: str):
    # Extract country from input
    country = extract_country_from_input(city)

    # Get currency info
    currency_code, currency_symbol = get_currency_for_country(country)

    # Create prompt with currency info
    prompt = create_itinerary_prompt(city, interests, currency_code, currency_symbol)

    response = llm.invoke(prompt.format_messages(
        city=city,
        interests=interests
    ))
    return response.content

# Beautiful full-page CSS with background image
custom_css = f"""
:root {{
    --primary: #4CAF50;
    --secondary: #FF5722;
    --text: #ffffff;
    --card-bg: rgba(30, 30, 30, 0.85);
    --input-bg: rgba(255, 255, 255, 0.1);
}}

body {{
    background: url('{random.choice([
        "https://images.unsplash.com/photo-1506929562872-bb421503ef21",
        "https://images.unsplash.com/photo-1464037866556-6812c9d1c72e",
        "https://images.unsplash.com/photo-1499678329028-101435549a4e",
        "https://images.unsplash.com/photo-1501785888041-af3ef285b470"
    ])}') no-repeat center center fixed;
    background-size: cover;
    min-height: 100vh;
    margin: 0;
    font-family: 'Poppins', sans-serif;
}}

.gradio-container {{
    max-width: 900px !important;
    margin: 0 auto !important;
    background: transparent !important;
    color: var(--text) !important;
    padding: 20px !important;
}}

.header {{
    text-align: center;
    padding: 30px;
    background: var(--card-bg);
    color: var(--text);
    border-radius: 15px;
    margin-bottom: 30px;
    backdrop-filter: blur(5px);
    border: 1px solid rgba(255,255,255,0.1);
}}

.header h1 {{
    font-size: 2.5rem;
    margin-bottom: 10px;
    background: linear-gradient(45deg, #4CAF50, #8BC34A);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
}}

.input-section {{
    background: var(--card-bg);
    padding: 30px;
    border-radius: 15px;
    margin-bottom: 30px;
    backdrop-filter: blur(5px);
    border: 1px solid rgba(255,255,255,0.1);
}}

.itinerary-output {{
    background: var(--card-bg) !important;
    color: var(--text) !important;
    padding: 30px;
    border-radius: 15px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.3);
    border-left: 5px solid var(--primary);
    margin-top: 20px;
    line-height: 1.8;
    backdrop-filter: blur(5px);
    border: 1px solid rgba(255,255,255,0.1);
}}

.generate-btn {{
    background: linear-gradient(45deg, var(--primary), var(--secondary)) !important;
    color: white !important;
    border: none !important;
    padding: 15px 35px !important;
    font-size: 1.1rem !important;
    border-radius: 50px !important;
    transition: all 0.3s ease !important;
    margin-top: 15px !important;
}}

.generate-btn:hover {{
    transform: translateY(-3px);
    box-shadow: 0 10px 20px rgba(0,0,0,0.2);
}}

input, textarea {{
    background: var(--input-bg) !important;
    color: var(--text) !important;
    border: 1px solid rgba(255,255,255,0.2) !important;
    padding: 12px 15px !important;
    border-radius: 8px !important;
}}

label {{
    color: var(--text) !important;
    font-size: 1.1rem !important;
    margin-bottom: 8px !important;
}}

.markdown h1, .markdown h2, .markdown h3 {{
    color: var(--text) !important;
    margin-top: 1.5em !important;
}}

.markdown ul, .markdown ol {{
    padding-left: 20px !important;
}}

.markdown li {{
    margin-bottom: 10px !important;
}}

@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700&display=swap');

@keyframes fadeIn {{
    from {{ opacity: 0; transform: translateY(20px); }}
    to {{ opacity: 1; transform: translateY(0); }}
}}

.itinerary-output {{
    animation: fadeIn 0.6s ease-out;
}}

.currency-notice {{
    background: rgba(76, 175, 80, 0.2);
    padding: 10px;
    border-radius: 8px;
    margin-bottom: 15px;
    border-left: 3px solid var(--primary);
}}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Default()) as demo:
    with gr.Column():
        gr.Markdown("""
        <div class="header">
            <h1>✈️ Wanderlust AI Planner</h1>
            <p style="font-size: 1.2rem; opacity: 0.9;">Get the itinerary for your perfect day trip</p>
        </div>
        """)

        with gr.Column(elem_classes="input-section"):
            city = gr.Textbox(
                label="📍 Destination (City, Country)",
                placeholder="Enter as 'City' or 'City, Country' (e.g., 'Tokyo' or 'Paris, France')",
                lines=1
            )
            interests = gr.Textbox(
                label="❤️ Your Interests",
                placeholder="What do you enjoy? (e.g., art, hiking, food markets, history)",
                lines=1
            )
            submit_btn = gr.Button(
                "✨ Generate Itinerary",
                variant="primary",
                elem_classes="generate-btn"
            )

        gr.Markdown("""
        <div class="currency-notice">
            💱 Prices will be shown in the local currency of your destination
        </div>
        """, visible=True)

        itinerary_output = gr.Markdown(
            elem_classes="itinerary-output",
            label="🌍 Your Personalized Itinerary"
        )

        gr.Examples(
            examples=[
                ["Tokyo, Japan", "anime, sushi, temples"],
                ["Paris, France", "art, croissants, photography"],
                ["Bali, Indonesia", "beaches, yoga, vegan food"],
                ["New York, USA", "broadway, museums, pizza"],
                ["Rome, Italy", "history, pasta, architecture"],
                ["Bangkok, Thailand", "street food, temples, markets"],
                ["Dubai, UAE", "luxury shopping, desert safari, skyscrapers"]
            ],
            inputs=[city, interests],
            label="🌎 Popular Destination Examples:",
            examples_per_page=5
        )

    submit_btn.click(
        fn=generate_itinerary,
        inputs=[city, interests],
        outputs=itinerary_output
    )

if __name__ == "__main__":
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://47d01bf168b406d3b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
